In [1]:
# general imports 
import os
from pathlib import Path
import hdmf
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 

# nwb specific imports 
import pynwb
from nwbwidgets import nwb2widget
from hdmf_zarr import NWBZarrIO 

In [2]:
# set data path
import sys
import platform
from os.path import join as pjoin

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_dir = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_dir = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_dir = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_dir = "/media/$USERNAME/Brain2025/"
    
print('data directory set to', data_dir)

data directory set to /data/


In [3]:
# Load metadata CSV 
metadata = pd.read_csv('/data/metadata/npultra_metadata.csv', index_col = False) 
metadata.head()

,project_name,_id,name,subject_id,genotype,date_of_birth,sex,modality,session_date,age,session_time,session_type,stimulus_epochs,stimulus_types,notes
0,NP Ultra and Psychedelics,916393f2-7009-421a-bde7-bbaf95416937,ecephys_714527_2024-05-14_13-22-07_nwb_2025-08...,714527,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-11-30,Female,Extracellular electrophysiology,2024-05-14,166,10:25:34.114677,saline,"['Spontaneous_0', 'RFMapping_0', 'OptoTagging_...",['Anesthesia' 'Injection' 'OptoTagging' 'RFMap...,['Saline injection occurred ~5 min after this ...
1,NP Ultra and Psychedelics,8a00ee0c-c470-43c5-b24c-52c1785bee3c,ecephys_714527_2024-05-15_13-00-23_nwb_2025-08...,714527,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-11-30,Female,Extracellular electrophysiology,2024-05-15,167,10:43:35.535979,psilocybin,"['Spontaneous_0', 'RFMapping_0', 'OptoTagging_...",['Anesthesia' 'Injection' 'OptoTagging' 'RFMap...,['Psilocybin injection (3 mg/kg) occurred ~1 m...
2,NP Ultra and Psychedelics,c867ad1e-bd84-42df-b863-1244a2fcc957,ecephys_714789_2024-05-16_13-16-59_nwb_2025-08...,714789,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-12-01,Female,Extracellular electrophysiology,2024-05-16,167,10:26:41.826355,saline,"['Spontaneous_0', 'RFMapping_0', 'OptoTagging_...",['Anesthesia' 'Injection' 'OptoTagging' 'RFMap...,['Saline injection occured ~1 min after this s...
3,NP Ultra and Psychedelics,48c07938-c41b-4cd6-b9e6-c25c0a90aedb,ecephys_714789_2024-05-17_11-47-07_nwb_2025-08...,714789,Sim1-Cre_KJ18/wt;Ai32(RCL-ChR2(H134R)_EYFP)/wt,2023-12-01,Female,Extracellular electrophysiology,2024-05-17,168,09:26:22.868269,psilocybin,"['Spontaneous_0', 'RFMapping_0', 'OptoTagging_...",['Anesthesia' 'Injection' 'OptoTagging' 'RFMap...,['Psilocybin injection (3 mg/kg) occurred ~1 m...
4,NP Ultra and Psychedelics,f6792a6f-e70b-4d42-b0b0-9007adc65775,ecephys_717033_2024-06-04_13-01-40_nwb_2025-08...,717033,Tlx3-Cre_PL56/wt;Ai167(TIT2L-ChrimsonR-tdT-ICL...,2023-12-18,Male,Extracellular electrophysiology,2024-06-04,169,10:11:23.881625,saline,"['Spontaneous_0', 'RFMapping_0', 'OptoTagging_...",['Anesthesia' 'Injection' 'OptoTagging' 'RFMap...,['Saline injection occurred ~1 minute after th...


In [4]:
# Select psilocybin sessions 
filtered_metadata = metadata[metadata.session_type == 'saline']

# Get the asset name 
session_name = filtered_metadata['name'].iloc[0]

# Find the filepath 
data_dir = os.path.join(r'/data/np-ultra-psychedelics/', session_name)
nwb_path_zarr = list(Path(data_dir).glob("*.nwb"))[0]

print('Found filepath:', nwb_path_zarr) 


Found filepath: /data/np-ultra-psychedelics/ecephys_714527_2024-05-14_13-22-07_nwb_2025-08-03_21-11-22/ecephys_714527_2024-05-14_13-22-07_experiment1_recording1.nwb


In [5]:
# Select psilocybin sessions 
filtered_metadata = metadata[metadata.session_type == 'psilocybin']

# Get the asset name 
session_name = filtered_metadata['name'].iloc[0]

# Find the filepath 
data_dir = os.path.join(r'/data/np-ultra-psychedelics/', session_name)
nwb_path_zarr = list(Path(data_dir).glob("*.nwb"))[0]

print('Found filepath:', nwb_path_zarr) 


Found filepath: /data/np-ultra-psychedelics/ecephys_714527_2024-05-15_13-00-23_nwb_2025-08-03_21-11-22/ecephys_714527_2024-05-15_13-00-23_experiment1_recording1.nwb


In [6]:
io = NWBZarrIO(nwb_path_zarr, mode = 'r') 
nwbfile_zarr = io.read()

In [7]:
nwb2widget(nwbfile_zarr)